# Course 4 - Project - Part 5: Logistic regression baseline

<a name="top-5"></a>
This notebook is concerned with *Part 5: Logistic regression baseline*.

**Contents:**
* [Step 0: Loading data](#step-5.0)
* [Step 1: Evaluate the logistic regression baseline](#step-5.1)
* [Step 2: Tune its regularization strength parameter](#step-5.2)
* [Step 3: Pick a few images and compute the probability for each class](#step-5.3)

## Step 0: Loading data<a name="step-5.0"></a> ([top](#top-5))
---

We load the training set with the extracted high-level features.

In [13]:
# Standard library.
import os
import pathlib
import typing as T

# 3rd party.
import numpy as np

# Project.
import utils

In [14]:
separator = ''.center(80, '-')

path_train = pathlib.Path.cwd() / 'data' / 'swissroads-features-train.npz'
data_train = utils.load(path_train)
print(separator)
print(f'Dataset: train\n{utils.info(data_train)}')

path_valid = pathlib.Path.cwd() / 'data' / 'swissroads-features-valid.npz'
data_valid = utils.load(path_valid)
print(separator)
print(f'Dataset: valid\n{utils.info(data_valid)}')

path_test = pathlib.Path.cwd() / 'data' / 'swissroads-features-test.npz'
data_test = utils.load(path_test)
print(separator)
print(f'Dataset: test\n{utils.info(data_test)}')

--------------------------------------------------------------------------------
Dataset: train
data: shape=(280, 224, 224, 3), dtype=float32
label_idxs: shape=(280,), dtype=int64
label_strs: shape=(6,), dtype=<U10
names: shape=(280,), dtype=<U19
features: shape=(280, 1280), dtype=float32
--------------------------------------------------------------------------------
Dataset: valid
data: shape=(139, 224, 224, 3), dtype=float32
label_idxs: shape=(139,), dtype=int64
label_strs: shape=(6,), dtype=<U10
names: shape=(139,), dtype=<U19
features: shape=(139, 1280), dtype=float32
--------------------------------------------------------------------------------
Dataset: test
data: shape=(50, 224, 224, 3), dtype=float32
label_idxs: shape=(50,), dtype=int64
label_strs: shape=(6,), dtype=<U10
names: shape=(50,), dtype=<U19
features: shape=(50, 1280), dtype=float32


In [15]:
label_strs = data_train['label_strs']  # Same for all data sets.
assert (
    np.all(data_train['label_strs'] == data_valid['label_strs']) and
    np.all(data_train['label_strs'] == data_test['label_strs'])
)

X_train = data_train['data']
y_train = data_train['label_idxs']
F_train = data_train['features']
N_train = data_train['names']

X_valid = data_valid['data']
y_valid = data_valid['label_idxs']
F_valid = data_valid['features']
N_valid = data_train['names']

X_test = data_test['data']
y_test = data_test['label_idxs']
F_test = data_test['features']
N_test = data_test['names']

## Step 1: Evaluate the logistic regression baseline<a name="step-5.1"></a> ([top](#top-5))
---

In [16]:
# Standard library.
import warnings

# 3rd party.
import pandas as pd
from sklearn.exceptions import ConvergenceWarning
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ParameterGrid
from sklearn.multiclass import OneVsOneClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

We want to use logistic regression for a multiclass classification task.

In this part it makes sense to merge the training and the validation sets since otherwise we would not make use of the validation set.

In [17]:
X_train_large = np.concatenate([X_train, X_valid])
y_train_large = np.concatenate([y_train, y_valid])
F_train_large = np.concatenate([F_train, F_valid])
N_train_large = np.concatenate([N_train, N_valid])

We create a pipeline comprised of an optional scaler and a logistic regression classifier (to be fleshed out later).

In [18]:
# Create the estimator.
logreg_pipe = Pipeline([
    ('scaler', None),
    ('logreg', None)
])

For the baseline, we decide to try:
* Scaling: none and standard-scaler.
* Strategies: one-vs-one (OVO), one-vs-rest (OVR) and softmax.

In [19]:
RANDOM_STATE = 0
MAX_ITER = 200

scalers = [None, StandardScaler()]

# Define the grid of values.
grid = ParameterGrid([
    # OVO
    {
        'scaler': scalers,
        'logreg': [
            OneVsOneClassifier(
                LogisticRegression(random_state=RANDOM_STATE)  # Make it deterministic.
            )
        ],
        'logreg__estimator__solver': ['liblinear']
    },
    # OVR
    {
        'scaler': scalers,
        'logreg': [
            LogisticRegression(random_state=RANDOM_STATE)  # Make it deterministic.
        ],
        'logreg__solver': ['liblinear'],
        'logreg__multi_class': ['ovr']
    },
    # Softmax
    {
        'scaler': scalers,
        'logreg': [
            LogisticRegression(random_state=RANDOM_STATE)  # Make it deterministic.
        ],
        'logreg__solver': ['saga'],
        'logreg__max_iter': [MAX_ITER],
        'logreg__multi_class': ['multinomial']
    }
])

**Note:** We had to work around a few warnings:
* The default maximum number of iterations for Softmax causes a `ConvergenceWarning: The max_iter was reached which means the coef_ did not converge`. So we increased this parameter from 100 (default) to 200 and suppressed the remaining warnings.

In [20]:
gs_results = []

with warnings.catch_warnings():

    # Filter convergence warnings.
    warnings.simplefilter('ignore', ConvergenceWarning)

    for params_dict in grid:
        # Set the parameters.
        logreg_pipe.set_params(**params_dict)

        # Fit the clasifier to the data.
        logreg_pipe.fit(F_train_large, y_train_large)

        # Store the scores.
        gs_result = params_dict
        gs_result['accuracy_train'] = logreg_pipe.score(F_train_large, y_train_large)
        gs_result['accuracy_test'] = logreg_pipe.score(F_test, y_test)

        # Save result.
        gs_results.append(gs_result)

We convert the results to a data frame.

In [21]:
# Convert results to a data frame.
df_results = (pd
              .DataFrame(gs_results)
              .sort_values(by='accuracy_test', ascending=False)
             )

In [22]:
# Make it more readable.
df_report = df_results.copy()
df_report['scaler'] = df_report['scaler'].apply(
    lambda est: (str(est) if est is None else
                 type(est).__name__).lower())
df_report['strategy'] = df_report['logreg'].apply(
    lambda est: ('ovo' if isinstance(est, OneVsOneClassifier) else
                 est.get_params()['multi_class']).lower())

df_report[['scaler', 'strategy', 'accuracy_train', 'accuracy_test']]

,scaler,strategy,accuracy_train,accuracy_test
3,standardscaler,ovr,1.0,0.98
2,none,ovr,1.0,0.94
5,standardscaler,multinomial,1.0,0.94
1,standardscaler,ovo,1.0,0.92
4,none,multinomial,1.0,0.92
0,none,ovo,1.0,0.90


**Comment:** So far, using a standard scaler and a logistic regression with OVR yields the best accuracy on the test set (98 %).

## Step 2: Tune its regularization strength parameter<a name="step-5.2"></a> ([top](#top-5))
---

In [23]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

We want to tune the regularization strength of the logistic regression classifier with cross-validated grid search.

In [25]:
RANDOM_STATE = 0
MAX_ITER = 200

scalers = [StandardScaler()]  # This should help the solvers.

# Using np.logspace(a, b, num=k * (b - a) + 1), where a, b are integers and k > 0 is an integer, we
# include integer exponents in the range [a, b]. 
Cs = np.logspace(-4, 4, num=3 * 8 + 1)  # C defaults to 1.0.

grid = [
    # OVO
    {
        'scaler': scalers,
        'logreg': [
            OneVsOneClassifier(
                LogisticRegression(random_state=RANDOM_STATE)  # Make it deterministic.
            )
        ],
        'logreg__estimator__C': Cs,
        'logreg__estimator__solver': ['liblinear']
    },
    # OVR
    {
        'scaler': scalers,
        'logreg': [
            LogisticRegression(random_state=RANDOM_STATE)  # Make it deterministic.
        ],
        'logreg__C': Cs,
        'logreg__solver': ['liblinear'],
        'logreg__multi_class': ['ovr']
    },
    # Softmax
    {
        'scaler': scalers,
        'logreg': [
            LogisticRegression(random_state=RANDOM_STATE)  # Make it deterministic.
        ],
        'logreg__C': Cs,
        'logreg__solver': ['saga'],
        'logreg__max_iter': [MAX_ITER],
        'logreg__multi_class': ['multinomial']
    }
]

cv = StratifiedKFold(n_splits=10, random_state=RANDOM_STATE)
logreg_gscv = GridSearchCV(logreg_pipe, grid, n_jobs=-1, iid=False, refit=False, cv=cv, return_train_score=True)

In [26]:
with warnings.catch_warnings():

    # Filter convergence warnings.
    warnings.simplefilter('ignore', ConvergenceWarning)

    # Fit/evaluate the estimator.
    logreg_gscv.fit(F_train_large, y_train_large);

In [ ]:
# Collect results in a data frame.
df_results = (pd
    .DataFrame({
        'strategy': np.where(~logreg_gscv.cv_results_['param_logreg__multi_class'].mask,
                              logreg_gscv.cv_results_['param_logreg__multi_class'], 'ovo'),
        'C': np.where(~logreg_gscv.cv_results_['param_logreg__C'].mask,
                       logreg_gscv.cv_results_['param_logreg__C'],
                       logreg_gscv.cv_results_['param_logreg__estimator__C']),
        'mean_train': logreg_gscv.cv_results_['mean_train_score'],
        'mean_test': logreg_gscv.cv_results_['mean_test_score'],
        'std_test': logreg_gscv.cv_results_['std_test_score'],
        'params': logreg_gscv.cv_results_['params']
    })
    .sort_values(by='mean_test', ascending=False)
)

### Best results for one-vs-one, one-vs-rest and softmax

Here are the 10 best combinations for one-vs-one, one-vs-rest and softmax (according to the mean test score).

In [ ]:
df_results[['strategy', 'C', 'mean_train', 'mean_test', 'std_test']].head(10)

**Comment:** The OVO strategy seems to yield the best result.

We compute the accuracy of the best model (among OVO, OVR and Softmax) on the test set.

In [ ]:
with warnings.catch_warnings():

    # Filter convergence warnings.
    warnings.simplefilter('ignore', ConvergenceWarning)

    params_best = df_results.iloc[0]['params']
    logreg_pipe.set_params(**params_best)
    logreg_pipe.fit(F_train_large, y_train_large)
    accuracy_train_large = logreg_pipe.score(F_train_large, y_train_large)
    accuracy_test = logreg_pipe.score(F_test, y_test)
    print(f'training accuracy: {accuracy_train_large * 100:.1f} %')
    print(f'test accuracy: {accuracy_test * 100:.1f} %')

### Best results for one-vs-rest and softmax only

here are the 10 best combinations for one-vs-rest and softmax only (according to the mean test score).

In [ ]:
df_results[df_results['strategy'] != 'ovo'].head(10)

In [ ]:
with warnings.catch_warnings():

    # Filter convergence warnings.
    warnings.simplefilter('ignore', ConvergenceWarning)

    params_best = df_results[df_results['strategy'] != 'ovo'].iloc[0]['params']
    logreg_pipe.set_params(**params_best)
    logreg_pipe.fit(F_train_large, y_train_large)
    accuracy_train_large = logreg_pipe.score(F_train_large, y_train_large)
    accuracy_test = logreg_pipe.score(F_test, y_test)
    print(f'training accuracy: {accuracy_train_large * 100:.1f} %')
    print(f'test accuracy: {accuracy_test * 100:.1f} %')

## Step 3: Pick a few images and compute the probability for each class<a name="step-5.3"></a> ([top](#top-5))
--- 

In [ ]:
# 3rd party.
%matplotlib inline
import matplotlib.pyplot as plt

Since only one-vs-rest and softmax offer probability estimates, we put one-vs-one aside.

We start by randomly picking 2 images in each class.

In [ ]:
NSAMPLES = 2

buf = []
for label in range(len(label_strs)):
    # Randomly choose indices of images that belong to this category.
    (idxs,) = np.nonzero(y_test == label)
    idxs_cat = np.random.choice(idxs, NSAMPLES, replace=False)
    buf.append(idxs_cat)
idxs_sample = np.concatenate(buf)
idxs_sample

In [ ]:
CELL_WIDTH = 1.75
CELL_HEIGHT = 1.75

# Make it deterministic.
np.random.seed(0)

nrows = 1
ncols = len(label_strs) * NSAMPLES

# Get the corresponding images, high-level features and names.
X_sample = X_test[idxs_sample]
F_sample = F_test[idxs_sample]  # shape=(1280,)
F_sample = F_sample[np.newaxis, :]  # shape=(1, 1280)
N_sample = N_test[idxs_sample]

fig, axs = plt.subplots(nrows=nrows, ncols=ncols,
                        figsize=(ncols * CELL_WIDTH, nrows * CELL_HEIGHT),
                        sharex=True, sharey=True)

# Display the images.
for j, (ax, idx) in enumerate(zip(axs, idxs_sample)):
    ax.imshow(X_test[idx])
    
    # Add labels.
    name_short = pathlib.Path(N_test[idx]).stem
    ax.set_xlabel(f'{name_short}', fontsize=10)

    # X/y-axis: hide the ticks and their labels.
    ax.tick_params(axis='both', which='both', length=0)
    ax.set_xticklabels([])
    ax.set_yticklabels([])

plt.show()
    
# Compute the probabilities.
# params_best
# logreg_pipe.set_params(**params_best)
# logreg_pipe.fit(F_train_large, y_train_large)
accuracy_train_large = logreg_pipe.score(F_train_large, y_train_large)
accuracy_test = logreg_pipe.score(F_test, y_test)
print(f'training accuracy: {accuracy_train_large * 100:.1f} %')
print(f'test accuracy: {accuracy_test * 100:.1f} %')

F_sample = F_test[idxs_sample]  # shape=(1280,)
probas_sample = logreg_pipe.predict_proba(F_sample)

In [ ]:
# The returned estimates for all classes are ordered by the label of classes.
short_names = [pathlib.Path(N_test[idx]).stem for idx in idxs_sample]
df = pd.DataFrame(data=probas_sample, index=short_names, columns=label_strs)


# https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html
df.style.highlight_max(axis=1).format('{:.2%}')

In [ ]:
params_best

In [ ]:
C_min = Cs.min()
C_max = Cs.max()

In [ ]:
def train_for_C(logreg_pipe, params, C, F_train, y_train, F_test, y_test, F_sample):
    # Compute the probabilities.
    params_C = params.copy()
    params_C['logreg__C'] = C
    params_C['logreg__max_iter'] = 1000
    logreg_pipe.set_params(**params_C)
    logreg_pipe.fit(F_train, y_train)
    
    accuracy_train = logreg_pipe.score(F_train, y_train)
    accuracy_test = logreg_pipe.score(F_test, y_test)
    print(f'C={C}')
    print(f'training accuracy: {accuracy_train:.2%}')
    print(f'test accuracy: {accuracy_test:.2%}')
    
    probas_sample = logreg_pipe.predict_proba(F_sample)
    # The returned estimates for all classes are ordered by the label of classes.
    short_names = [pathlib.Path(N_test[idx]).stem for idx in idxs_sample]
    df = pd.DataFrame(data=probas_sample, index=short_names, columns=label_strs)

    # https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html
    return df.style.highlight_max(axis=1).format('{:.2%}')

In [ ]:
train_for_C(logreg_pipe, params_best, Cs.min(), F_train, y_train, F_test, y_test, F_sample)

In [ ]:
train_for_C(logreg_pipe, params_best, Cs.max(), F_train, y_train, F_test, y_test, F_sample)